In [ ]:
import airfoil_simulator
import tensorflow as tf
import numpy as np
import lstm_model
import time
import skopt 
import visualize as viz

In [ ]:
def obj_airfoil_lift_drag(x):
    x = np.array(x)
    obj_value = -1*airfoil_simulator.objective(x.reshape(-1))/100
    return obj_value

In [ ]:
# imp.reload(lstm_model) # remove this when it's ready
steps = 20
dim = 2
x_0 = np.array([-0.1,-0.1]).reshape(-1,2)

In [ ]:
def optimize_lstm(x_0, steps):
    #load best model
    best_model_2d='rnn-cell-2d-1498485717'   
    print("Starting at %s and optimize for %d steps" % (x_0, steps) )
    sess, model_params = lstm_model.load_trained_model(best_model_2d)
    
    start_time = time.time()
    samples_x, samples_y = lstm_model.generate_sample_sequence(sess, model_params, x_0, steps = steps, \
        obj_func=lambda x: np.array(obj_airfoil_lift_drag(x)).reshape(1,-1) \
    )

    samples_y = np.array(samples_y).flatten()

    #print("Min -L/D found %f at %s" % (samples_y[min_y_idx], samples_x[min_y_idx].flatten()) )
    print("Took %f mins" % (( time.time() - start_time ) / 60.0) )
    
    return samples_x, samples_y

def optimize_random(x_0, steps):
    res = skopt.dummy_minimize(obj_airfoil_lift_drag, dimensions=[(-1.0,1.0)]*2, n_calls=steps, 
                               x0=x_0, verbose=True) 
    return res.xiters, res.funcvals

def optimize_skopt(x_0, steps):
    res = skopt.gp_minimize(obj_airfoil_lift_drag, dimensions=[(-1.0,1.0)]*2, n_calls=steps, 
                            n_random_starts=10, x0=x_0, verbose=True) 
    return res.xiters, res.funcvals

In [ ]:
samples_x_random, samples_y_random = optimize_random(x_0, steps)
samples_x_skopt, samples_y_skopt = optimize_skopt(x_0, steps)
samples_x_lstm, samples_y_lstm = optimize_lstm(x_0, steps)

In [ ]:
viz.plot_min_curves([samples_y_lstm[np.newaxis,:], np.array(samples_y_random)[np.newaxis,:],
                    np.array(samples_y_skopt)[np.newaxis,:]],['MIN', 'RANDOM','SKOPT'])